## Задача о движении 2 небесных тел в СС

Задача о движении 2 небесных тел является распростанненой задачей механики. 

Пусть есть 2 тела, имеющих массы $m_1$ и $m_2$ соотвественно. Напомним Второй закон Ньютона
$$F = ma,$$
где $a$ это ускорение. Тогда сила притяжения тела 1 на тело 2 запишется в виде
$$F_{21}=\frac{Gm_{1}m_{2}}{r^3_{21}}r_{21},$$
где $r_{21}$ это вектор расстояния между двумя телам. Сила, действующая на из тела 2 на тело 1, по Третьему закону  Ньютона равна $F_{12}=-F_{21}$, а расстояние $r_{12}=-r_{21}$. 
Данная задача описывается следующей системой дифференциальных уравнений: $$\begin{cases}
x^{''}_1(t)=\frac{Gm_2(x_2-x_1)}{|r|^3}\\
y^{''}_1(t)=\frac{Gm_2(y_2-y_1)}{|r|^3}\\
x^{''}_2(t)=\frac{Gm_1(x_1-x_2)}{|r|^3}\\
y^{''}_2(t)=\frac{Gm_1(y_1-y_2)}{|r|^3}\\
\end{cases}$$

Данную систему можно свести к систему из 8 ДУ первого порядка и решить численно с помощью Метода Рунге-Кутта.

Класс для решения СДУ методом Рунге-Кутта 4 порядка:

In [ ]:
from numpy import *

# The gravitational constant G
G = 6.67428e-11

class RK4():
    def __init__(self, r1, r2, v1, v2, step, m1, m2):
        self.r1 = r1
        self.r2 = r2
        self.v1 = v1
        self.v2 = v2
        self.h = step
        self.m1 = m1
        self.m2 = m2
        self.t = 0
        
    def getR1(self):
        return self.r1
    
    def getR2(self):
        return self.r2
    
    def getV1(self):
        return self.v1
    
    def getV2(self):
        return self.v2
    
    def getH(self):
        return self.h
    
    def getM1(self):
        return self.m1
    
    def getT(self):
        return self.t
    
    ############
    def __gravityForce(self, s1, s2, mass):
        return ((s2 - s1) * G * mass) / (linalg.norm(s2 - s1) ** 3)
    
    def __getAccBody(self, r1, r2, m):
        return ((r2 - r1) * G * m) / (linalg.norm(r2 - r1)**3) # Ускорение первого тела
    ############
    
    def runIter(self):
        g1 = self.v1
        g2 = self.v1 + g1 * 0.5 * self.h
        g3 = self.v1 + g2 * 0.5 * self.h
        g4 = self.v1 + g3 * self.h
        c = (g1 + (g2 * 2) + (g3 * 2) + g4) * self.h / 6.0 + self.r1
        
        a1 = self.__getAccBody(self.r1, self.r2, self.m2)
        a2 = self.__getAccBody(self.r2, self.r1, self.m1)
        
        h1 = self.v2
        h2 = self.v2 + h1 * 0.5 * self.h
        h3 = self.v2 + h2 * 0.5 * self.h
        h4 = self.v2 + h3 * self.h
        d = (h1 + h2 * 2 + h3 * 2 + h4) * self.h / 6.0 + self.r2

        k1 = self.__gravityForce(self.r1, self.r2, self.m2)
        k2 = self.__gravityForce(self.r1 + g1 * 0.5 * self.h, self.r2 + h1 * 0.5 * self.h, self.m2)
        k3 = self.__gravityForce(self.r1 + g2 * 0.5 * self.h, self.r2 + h2 * 0.5 * self.h, self.m2)
        k4 = self.__gravityForce(self.r1 + g2 * self.h, self.r2 + h2 * self.h, self.m2)
        e = (k1 + k2 * 2 + k3 * 2 + k4) * self.h / 6.0 + self.v1 # Скорость первого тела

        l1 = k1 * (-1)
        l2 = k2 * (-1)
        l3 = k3 * (-1)
        l4 = k4 * (-1)
        f = (l1 + l2 * 2 + l3 * 2 + l4) * self.h / 6.0 + self.v2 #Скорость второго тела

        self.r1 = c
        self.r2 = d
        self.v1 = e
        self.v2 = f
        self.t += self.h

Пример численного решения:

In [ ]:
%%time
TBP = RK4(array([0, 0]), array([384000000, 0]), array([0,0]), array([-10, 10]),
                 0.01, 73400000000000000000000.0, 6000000000000000000000000.0)
e = [[],[]]
c = [[],[]]
for i in range(100000):
    print('t = %s' % (TBP.getT()))
    print('Earth x = %s, y = %s' %(TBP.getR1()[0], TBP.getR1()[1]))
    print('Moon x = %s, y = %s' %(TBP.getR2()[0], TBP.getR2()[1]))
    print('dX = %s, dY = %s' %(TBP.getR2()[0]-TBP.getR1()[0], TBP.getR2()[1]-TBP.getR1()[1]))
    TBP.runIter()

Класс для построения анимации движения тел относительно Солнца:

In [ ]:
import math
from turtle import *

G = 6.67428e-11
AU = (149.6e6 * 1000)
SCALE = 250 / AU

class Body(Turtle):

    name = 'Body'
    mass = None
    vx = vy = 0.0
    px = py = 0.0
    
    def attraction(self, other):
        if self is other:
            raise ValueError("Attraction of object %r to itself requested"
                             % self.name)

        
        sx, sy = self.px, self.py
        ox, oy = other.px, other.py
        dx = (ox-sx)
        dy = (oy-sy)
        d = math.sqrt(dx**2 + dy**2)

        if d == 0:
            raise ValueError("Collision between objects %r and %r"
                             % (self.name, other.name))
        
        f = G * self.mass * other.mass / (d**2)

        theta = math.atan2(dy, dx)
        fx = math.cos(theta) * f
        fy = math.sin(theta) * f
        return fx, fy

def update_info(step, bodies):
    print('Step #{}'.format(step))
    for body in bodies:
        s = '{:<8}  Pos.={:>6.2f} {:>6.2f} Vel.={:>10.3f} {:>10.3f}'.format(
            body.name, body.px/AU, body.py/AU, body.vx, body.vy)
        print(s)
    print()

def loop(bodies):
    timestep = 24*3600  # 1 day
    
    for body in bodies:
        body.penup()
        body.hideturtle()

    step = 1
    while True:
        update_info(step, bodies)
        step += 1

        force = {}
        for body in bodies:
            total_fx = total_fy = 0.0
            for other in bodies:
                if body is other:
                    continue
                fx, fy = body.attraction(other)
                total_fx += fx
                total_fy += fy

            force[body] = (total_fx, total_fy)

        for body in bodies:
            fx, fy = force[body]
            body.vx += fx / body.mass * timestep
            body.vy += fy / body.mass * timestep
            body.px += body.vx * timestep
            body.py += body.vy * timestep
            body.goto(body.px*SCALE, body.py*SCALE)
            body.dot(3)


def main():
    sun = Body()
    sun.name = 'Sun'
    sun.mass = 1.98892 * 10**30
    sun.pencolor('yellow')

    earth = Body()
    earth.name = 'Earth'
    earth.mass = 5.9742 * 10**24
    earth.px = -1*AU
    earth.vy = 29.783 * 1000            # 29.783 km/sec
    earth.pencolor('blue')

    venus = Body()
    venus.name = 'Venus'
    venus.mass = 4.8685 * 10**24
    venus.px = 0.723 * AU
    venus.vy = -35.02 * 1000
    venus.pencolor('red')

    loop([sun, earth, venus])

In [ ]:
main()

Скриншоты работы программы:

![T_start](shm1.png)

![T_end](shm2.png)

Программа для построения анимации движения Земли и Венеры относительно Солнца(необходима установка библиотек Tkinter и Turtle https://yadi.sk/d/c5iK3NTK3RsQt4